In [1]:
import pathlib
from sklearn import neighbors
from sklearn import metrics
import string
import numpy as np
from tqdm.auto import tqdm

import spacy
nlp = spacy.load('en_core_web_lg')

from scipy import spatial

C:\Users\Sam's PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
docs = []
labels = []

directory = pathlib.Path(r'..\data\genderData')
label_names = ['Male', 'Female']

for label in tqdm(label_names):
    for file in tqdm(directory.joinpath(label).iterdir()):
        labels.append(label)
        docs.append(file.read_text(encoding='utf-8'))

  0%|          | 0/2 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [3]:
doc = docs[0]
print(labels[0])
doc

Male


'WASHINGTON  —   Congressional Republicans have a new fear when it comes to their    health care lawsuit against the Obama administration: They might win. The incoming Trump administration could choose to no longer defend the executive branch against the suit, which challenges the administration’s authority to spend billions of dollars on health insurance subsidies for   and   Americans, handing House Republicans a big victory on    issues. But a sudden loss of the disputed subsidies could conceivably cause the health care program to implode, leaving millions of people without access to health insurance before Republicans have prepared a replacement. That could lead to chaos in the insurance market and spur a political backlash just as Republicans gain full control of the government. To stave off that outcome, Republicans could find themselves in the awkward position of appropriating huge sums to temporarily prop up the Obama health care law, angering conservative voters who have been 

In [4]:
def clean_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.replace('\n', ' ')
    text = ' '.join(text.split())  # remove multiple whitespaces
    return text

In [5]:
doc = clean_text(doc)
doc

'washington — congressional republicans have a new fear when it comes to their health care lawsuit against the obama administration they might win the incoming trump administration could choose to no longer defend the executive branch against the suit which challenges the administration’s authority to spend billions of dollars on health insurance subsidies for and americans handing house republicans a big victory on issues but a sudden loss of the disputed subsidies could conceivably cause the health care program to implode leaving millions of people without access to health insurance before republicans have prepared a replacement that could lead to chaos in the insurance market and spur a political backlash just as republicans gain full control of the government to stave off that outcome republicans could find themselves in the awkward position of appropriating huge sums to temporarily prop up the obama health care law angering conservative voters who have been demanding an end to the

In [6]:
def embed(tokens, nlp):
    """Return the centroid of the embeddings for the given tokens.

    Out-of-vocabulary tokens are cast aside. Stop words are also
    discarded. An array of 0s is returned if none of the tokens
    are valid.

    """

    lexemes = (nlp.vocab[token] for token in tokens)

    vectors = np.asarray([
        lexeme.vector
        for lexeme in lexemes
        if lexeme.has_vector
        and not lexeme.is_stop
        and len(lexeme.text) > 1
    ])

    if len(vectors) > 0:
        centroid = vectors.mean(axis=0)
    else:
        width = nlp.meta['vectors']['width']  # typically 300
        centroid = np.zeros(width)

    return centroid

In [7]:
tokens = doc.split(' ')
centroid = embed(tokens, nlp)
print(centroid.shape)
print(centroid[:10])

(300,)
[-1.0341872   0.29092434 -2.5468051   0.5357483   2.9095006   0.71387833
  0.8422483   3.4557972  -0.9317664  -0.9081056 ]


In [8]:
label_vectors = np.asarray([
    embed(label.split(' '), nlp)
    for label in label_names
])
label_vectors.shape

(2, 300)

In [9]:
neigh = neighbors.NearestNeighbors(n_neighbors=1)
neigh.fit(label_vectors)
closest_label = neigh.kneighbors([centroid], return_distance=False)[0, 0]
label_names[closest_label]

'Female'

In [10]:
def predict(doc, nlp, neigh):
    doc = clean_text(doc)
    tokens = doc.split(' ')[:50]
    centroid = embed(tokens, nlp)
    closest_label = neigh.kneighbors([centroid], return_distance=False)[0][0]
    return closest_label

In [11]:
neigh = neighbors.NearestNeighbors(
    n_neighbors=1,
    metric=spatial.distance.cosine
)

neigh.fit(label_vectors)

preds = [label_names[predict(doc, nlp, neigh)] for doc in tqdm(docs)]

report = metrics.classification_report(
    y_true=labels,
    y_pred=preds,
    labels=label_names
)

print(report)

  0%|          | 0/40666 [00:00<?, ?it/s]

C:\Users\Sam's PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\spatial\distance.py:620: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


              precision    recall  f1-score   support

        Male       0.35      0.00      0.00     18727
      Female       0.54      1.00      0.70     21939

    accuracy                           0.54     40666
   macro avg       0.45      0.50      0.35     40666
weighted avg       0.45      0.54      0.38     40666

